In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd 

r = requests.get('https://www.youtube.com/playlist?list=PLmmc_Sy1OV2JpAOi6RGd4sB4npoZnCtVn')
soup = BeautifulSoup(r.text,'html.parser')

title_results = soup.find_all('a', attrs={'class':'pl-video-title-link'})
timestamp_results = soup.find_all('div', attrs={'class':'timestamp'})

timestamps = []
seconds = []
titles = []


for ts_result in timestamp_results:
    ts = ts_result.find("span")
    ts = str(ts.contents[0])
    sc = (int(re.split(':',ts)[0])*60)+(int(re.split(':',ts)[1]))
    seconds.append(sc)
    timestamps.append(ts)

    
for tl_result in title_results:
    tl = tl_result.contents[0].strip()
    titles.append(tl)

    
records = {'Title':titles,'Length':timestamps,'Seconds':seconds}


df = pd.DataFrame(records)
df['RT'] = df['Seconds'].cumsum()


df['H'] = df['RT'] // (60*60)
df['M'] = (df['RT'] - (df['H']*60*60) )  // 60
df['S'] = df['RT'] % 60

df['Running Length'] = df['H'].apply(str).str.zfill(2)+':'+df['M'].apply(str).str.zfill(2)+':'+df['S'].apply(str).str.zfill(2)

total_duration = df['RT'].iat[-1]
df['RL'] = total_duration - df['RT']

df['H1'] = df['RL'] // (60*60)
df['M1'] = (df['RL'] - (df['H1']*60*60) )  // 60
df['S1'] = df['RL'] % 60

df['Remaining Time'] = df['H1'].apply(str).str.zfill(2)+':'+df['M1'].apply(str).str.zfill(2)+':'+df['S1'].apply(str).str.zfill(2)

df2 = df.drop(['Seconds','RT','H','M','S','RL','H1','M1','S1'], axis=1)

df2.to_csv('playlist_time.csv', index=False, encoding='utf-8') 

df2